<a href="https://colab.research.google.com/github/flexinJ/flexin/blob/main/Flexin_Analyzer_%2424_updates_october.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installation of dependencies

In [1]:
!pip install yfinance
!pip install pandas_ta
!pip install hurst
!pip install requests

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.1/115.1 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pandas_ta: filename=pandas_ta-0.3.14b0-py3-none-any.whl size=218908 sha256=686374a6f17091afdc28bfe46fc1ba9274c8b88c73d97c7dc3e3d0200269ba7a
  Stored in directory: /root/.cache/pip/wheels/69/00/ac/f7fa862c34b0e2ef320175100c233377b4c558944f12474cf0
Successfully built pandas_ta


Initial variables

In [2]:
wallet = ["CL=F",
          "AUDJPY=X",
          "JPY=X",
          "NZDUSD=X",
          "AUDUSD=X",
          "CAD=X",
          "GBPUSD=X",
          "EURUSD=X",
          "CHF=X"]
history = "1wk"
time = ["30m", "5m", "1m"]

Connects telegram API

In [3]:
def telegram(message):
  import requests
  apiToken = "6183952601:AAFbfrKfZb3iKX6GfYZQnatxo0lg1S4lL8Y"
  chatID = "@flexinflexinbot1"
  apiURL = f"https://api.telegram.org/bot{apiToken}/sendMessage"
  try:
    response = requests.post(apiURL, json={"chat_id": chatID, "text": message})
    print(f"📲 The message was sent to telegram...")
    return response
  except:
    return f"❌ Error sending message to telegram..."

Collect financial data

In [4]:
def get(symbol, history: str, time: str):
  import yfinance as yf
  try:
    df = yf.Ticker(symbol).history(period=history, interval=time)
    df["Symbol"] = symbol
    df["Time"] = time
    show_time(time)
    return df
  except:
    return f"❌ Unable to collect data for {symbol}..."

Indicators

In [5]:
def adx(df):
  import pandas_ta as ta
  try:
    adx = ta.adx(df["High"], df["Low"], df["Close"], length=14)
    df = df.join(adx)
    return df
  except:
    return f"❌ Error generating ADX..."

In [6]:
def macd(df):
  import pandas_ta as ta
  try:
    macd = ta.macd(df["Close"])
    df = df.join(macd)
    return df
  except:
    return f"❌ Error generating MACD..."

In [7]:
def atr(df):
  import pandas_ta as ta
  try:
    atr = ta.atr(df["High"], df["Low"], df["Close"], length=14)
    df = df.join(atr)
    return df
  except:
    return f"❌ Error generating ATR..."

In [8]:
def hurst(df):
  from hurst import compute_Hc
  try:
    def H(x): return compute_Hc(x)[0]
    df["Hurst"] = df["Close"].rolling(window=100).apply(H)
    return df
  except:
    return f"❌ Error generating HURST.."

In [9]:
def fast(df):
  import pandas_ta as ta
  try:
    fast = ta.ema(df["Close"], 50)
    df = df.join(fast)
    return df
  except:
    return f"❌ Error generating EMA 50.."

In [10]:
def slow(df):
  import pandas_ta as ta
  try:
    slow = ta.ema(df["Close"], 200)
    df = df.join(slow)
    return df
  except:
    return f"❌ Error generating EMA 200..."

In [11]:
def fft(df):
  import numpy as np
  try:
    fft = np.fft.fft(df["Close"].values)
    index = np.argsort(np.abs(fft))[::-1]
    filter = np.zeros_like(fft)
    filter[index[:1]] = fft[index[:1]]
    ifft = np.fft.ifft(filter).real
    return ifft[-1]
  except:
    return f"❌ Error generating FFT..."

Signal Check

In [12]:
def adxSignal(df):
  try:
    df.loc[(df["ADX_14"].shift(1) > 25) & (df["DMP_14"].shift(1) > df["DMN_14"].shift(1)), "ADX Signal"] = "Uptrend"
    df.loc[(df["ADX_14"].shift(1) > 25) & (df["DMN_14"].shift(1) > df["DMP_14"].shift(1)), "ADX Signal"] = "Downtrend"
    return df
  except:
    return f"❌ Error generating ADX signal..."

In [13]:
def macdSignal(df):
  try:
    df.loc[df["MACDh_12_26_9"].shift(1) > 0, "MACD Signal Hist"] = "Buy"
    df.loc[df["MACDh_12_26_9"].shift(1) < 0, "MACD Signal Hist"] = "Sell"
    return df
  except:
    return f"❌ Error generating MACD signal..."

In [14]:
def signal(df):
  try:
    df.loc[(df["Hurst"].shift(1) > 0.55) & (df["MACD Signal Hist"].shift(1) == "Buy") & (df["ADX Signal"].shift(1) == "Uptrend") & (df["EMA_50"].shift(1) > df["EMA_200"].shift(1)) & (df["Close"].shift(1) > df["EMA_50"].shift(1)), "Signal"] = "Buy"
    df.loc[(df["Hurst"].shift(1) > 0.55) & (df["MACD Signal Hist"].shift(1) == "Sell") & (df["ADX Signal"].shift(1) == "Downtrend") & (df["EMA_50"].shift(1) < df["EMA_200"].shift(1)) & (df["Close"].shift(1) < df["EMA_50"].shift(1)), "Signal"] = "Sell"
    return df
  except:
    return f"❌ Error generating Final signal..."

Apply indicators and signal checkers

In [31]:
def apply(symbol, history, time):
  try:
    df = get(symbol, history, time)
    df = adx(df)
    df = macd(df)
    df = fast(df)
    df = slow(df)
    df = hurst(df)
    df = atr(df)
    df = adxSignal(df)
    df = macdSignal(df)
    df = signal(df)
    print(" |🌧️ Indicators applied")
    return df
  except:
    return f"❌ Error apply indicators..."

Collect trade information

In [32]:
def compute_atr(symbol, history):
  time = "30m"
  data = get(symbol, history, time)
  data = atr(data)
  return data

In [33]:
def buy(df):
  try:
    symbol = df["Symbol"][-1]
    type = df["Signal"][-1]
    price = df["Close"][-1]
    atr = df["ATRr_14"][-1]
    tp = price+atr
    sl = price-(1.5*atr)
    telegram(f"""
    🫡 Flexin Analyzer 🤳
    Symbol: {symbol}
    Type: {type}
    EP: {price.round(4)}
    TP: {tp.round(4)}
    SL: {sl.round(4)}
              """)
  except:
    return f"❌ Error collecting trade information..."

In [34]:
def sell(df):
  try:
    symbol = df["Symbol"][-1]
    type = df["Signal"][-1]
    price = df["Close"][-1]
    atr = df["ATRr_14"][-1]
    tp = price-atr
    sl = price+(1.5*atr)
    telegram(f"""
    🫡 Flexin Analyzer 🤳
    Symbol: {symbol}
    Type: {type}
    EP: {price.round(4)}
    TP: {tp.round(4)}
    SL: {sl.round(4)}
              """)
  except:
    return f"❌ Error collecting trade information..."

Console Log

In [35]:
def show_buy():
  return print("📈 Buy")

In [36]:
def show_sell():
  return print("📉 Sell")

In [37]:
def wait():
  return print("💤 No signal")

Signal checker

In [38]:
def check(df):
  if df["Signal"][0] == df["Signal"][1] == df["Signal"][2]:
    if df["Signal"][0] == "Buy" and df["Signal"][1] == "Buy" and df["Signal"][2] == "Buy":
      show_buy()
      order = "Buy"
      return order
    elif df["Signal"][0] == "Sell" and df["Signal"][1] == "Sell" and df["Signal"][2] == "Sell":
      show_sell()
      order = "Sell"
      return order
  else:
    order = None
    return order

Display

In [39]:
def title(symbol):
  return print(f"🕵️ Analyzing {symbol}...")

In [40]:
def show_time(time):
  return print(f"⌚ {time}")

In [41]:
def show_hurst(df):
  H = df["Hurst"][-1]
  return print(f" |🌱 {H.round(2)}")

Tools

In [42]:
def transition():
  from IPython.display import clear_output
  from time import sleep
  sleep(60)
  clear_output()

Flexin Analyzer

In [43]:
def flexin(symbols, history, times):
  import pandas as pd
  import concurrent.futures as cf
  from concurrent.futures import ThreadPoolExecutor

  for symbol in symbols:
    result = list()
    title(symbol)
    for time in times:
      df = apply(symbol, history, time)
      show_hurst(df)
      result.append({
          "Symbol": symbol,
          "Time": time,
          "Signal": df["Signal"][-1],
          "Price": df["Close"][-1],
          "ATR": df["ATRr_14"][-1]
          })
    signal = pd.DataFrame(result)
    try:
      if check(signal) == "Buy":
        telegram(result)
        #data = compute_atr(symbol, history)
      elif check(signal) == "Sell":
        telegram(result)
        #data = compute_atr(symbol, history)
      else:
        wait()
        transition()
    except:
      return f"Check Signal no response for {symbol}... ❌"

Main function

In [44]:
def main():
  while True:
    flexin(wallet, history, time)
    transition()

Execute

In [ ]:
main()

Thanks :)